In [ ]:
#Peter Adranly
#I imported the polaris dataset, removed the NaN or missing values, and examined the shape.
#Next, I converted the CXSMILES data into 3D conformers.
#Finally, I built a Random Forest model with the MSE.
#The most difficult part of this assignment was the 2D molecules to 3D conformer conversions.
#This part proved tricky as it is where the tools I was the least familiar with were implemented.

In [7]:
from openeye import oechem,oequacpac,oeomega,oeshape, oezap
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error 
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sys import exit
from IPython import display

In [9]:
print("Dataset Loading")
print("________________")
dataset = pd.read_csv("antiviral_potency_2025_unblinded.csv")
dataset.shape
print(dataset)

print("NaNs in dataset")
print("________________")
none = dataset.isna().sum()
print(none)

print("Removing NaN values")
print("____________________________________")
clean = dataset.dropna()
clean.shape
clean

Dataset Loading
________________
                                               CXSMILES Molecule Name    Set  \
0     COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...  ASAP-0000141  Train   
1     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...  ASAP-0000142  Train   
2     CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...  ASAP-0000143  Train   
3     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...  ASAP-0000144  Train   
4     C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...  ASAP-0000145  Train   
...                                                 ...           ...    ...   
1323  O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...  ASAP-0032561   Test   
1324  O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...  ASAP-0032562   Test   
1325  O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...  ASAP-0032572   Test   
1326  COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...  ASAP-0032604   Test   
1327  COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...  ASAP-0032605   Test   

      

,CXSMILES,Molecule Name,Set,pIC50 (MERS-CoV Mpro),pIC50 (SARS-CoV-2 Mpro)
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,ASAP-0000142,Train,4.92,5.29
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,ASAP-0000144,Train,4.90,6.11
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,ASAP-0000145,Train,4.81,5.62
5,C=C(CN(C(=O)C1CCOC2=C1C=C(Cl)C=C2)C1=CN=CC2=CC...,ASAP-0000146,Train,4.88,6.45
6,C=C(CN(C(=O)C1CCOC2=C1C=C(Cl)C=C2)C1=CN=CC2=CC...,ASAP-0000147,Train,4.81,5.56
...,...,...,...,...,...
1322,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,ASAP-0032560,Test,4.33,4.02
1323,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,ASAP-0032561,Test,4.54,4.20
1325,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...,ASAP-0032572,Test,4.84,5.18
1326,COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...,ASAP-0032604,Test,5.53,5.59


In [11]:
omega_opts = oeomega.OEOmegaOptions()
omega_opts.SetMaxConfs(1)
omega_opts.SetEnergyWindow(5.0)
omega_opts.SetStrictStereo(True)             
omega_opts.SetSampleHydrogens(False)
omega_opts.SetCanonOrder(True)
omega_opts.SetRMSThreshold(0.5)
omega = oeomega.OEOmega(omega_opts)

print("Processing 2d to 3d molecules")
print("______________________________")
results = []

for idx, row in clean.iterrows():
    try:
        cxsmiles = row["CXSMILES"]
        mol = oechem.OEMol()
        if not oechem.OESmilesToMol(mol, cxsmiles):
            continue
            
        if mol.NumAtoms() > 80:
            print("Skipping large molecule:", row["Molecule Name"])
            continue

        oechem.OEAddExplicitHydrogens(mol)

        ret = omega.Build(mol)
        if ret != oeomega.OEOmegaReturnCode_Success:
            continue

        volume = oeshape.OECalcVolume(mol)
        pmi = oechem.OEDoubleArray(3)
        oechem.OECalcPMI(pmi, mol)

        results.append({
            "Molecule Name": row["Molecule Name"],
            "Volume": volume,
            "PMI_X": pmi[0],
            "PMI_Y": pmi[1],
            "PMI_Z": pmi[2],
            "pIC50_MERS": row["pIC50 (MERS-CoV Mpro)"],
            "pIC50_SARS2": row["pIC50 (SARS-CoV-2 Mpro)"]
        })

        del mol

        if idx % 60 == 0:
            print("Processed:", idx)
    except Exception as e:
        print("Failed index:", idx)
        continue

features_3D = pd.DataFrame(results)
features_3D.to_csv("antiviral_3D_features.csv", index=False)

Processed: 30
Processed: 60
Processed: 90
Processed: 120
Processed: 150
Processed: 180
Processed: 210
Processed: 300
Processed: 330
Processed: 360
Processed: 390
Processed: 420
Processed: 450
Processed: 480
Processed: 510
Processed: 540
Processed: 570
Processed: 600
Processed: 690
Processed: 720
Processed: 810
Processed: 930
Processed: 960
Processed: 990
Processed: 1020
Processed: 1050
Processed: 1080
Processed: 1110
Processed: 1140
Processed: 1170
Processed: 1230
Processed: 1260
Processed: 1290
Processed: 1320


In [19]:
print("Random Forest Model Building")
print("_____________________________")
model_df = features_3D.drop(columns=["Molecule Name"])

X = model_df.drop(columns=["pIC50_MERS"])
y = model_df["pIC50_MERS"]

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

model.fit(train_X, train_y)

pred = model.predict(test_X)

print(pred)

print("Error Metrics")
print("______________
print("R2:", r2_score(test_y, pred))
print("RMSE:", np.sqrt(mean_squared_error(test_y, pred)))

[4.58166667 4.44436667 4.2367     7.18823333 4.95103333 4.29463333
 4.2116     6.29493333 4.8477     6.21886667 4.89023333 4.33036667
 4.54616667 4.2295     5.10073333 5.9606     5.57363333 5.21643333
 6.35986667 5.08853333 4.19946667 4.17693333 5.28166667 4.2453
 4.825      4.0605     4.70343333 4.42766667 4.85236667 5.08236667
 4.5983     4.69973333 4.82586667 4.79713333 5.09163333 6.35756667
 6.1575     5.34503333 4.58043333 4.90086667 5.6141     6.2151
 5.47553333 4.33796667 4.56876667 5.36903333 4.62543333 4.61886667
 4.73413333 4.27286667 4.2409     5.45643333 4.1188     4.9047
 5.0311     4.2282     3.97706667 5.6718     5.86373333 4.36066667
 4.52066667 5.97053333 5.92363333 4.1395     4.39146667 6.57936667
 5.15153333 6.24113333 4.36746667 5.26366667 6.54366667 5.237
 4.90086667 4.43713333 5.13536667 4.36273333 5.26366667 5.7212
 4.13516667 5.10126667 5.0336     6.2756     5.06806667 4.2471
 4.6488     6.28786667 4.24246667 5.62566667 4.50646667 4.30323333
 4.1509     7.2131  